## Llama Index and Llama 2 tutorial on Lonestar 6

Llama2 is the Meta open source Large Language Model. LlamaIndex is a python library that connects data to the LLMs such as Llama2. This allows the user to quickly use their unstructured data as a basis for any chats or outputs. 


In [1]:
import os
import logging
import sys
from IPython.display import Markdown, display

from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate, PromptType
from pathlib import Path
from llama_index import download_loader, KnowledgeGraphIndex,  Document, SummaryIndex
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import re

In [2]:
import torch

## Set your working directory
Change your working directory to your Scratch location. This will improve performance, and ensure you have access to the model you rsynced earlier

In [3]:
scratch = ! echo $SCRATCH
os.chdir(scratch[0])
! pwd


/scratch/06659/wmobley


## Access the model
Next we'll access the models. You have 4 models to access the 7 and 13billion parameters chat and normal model. The folder will also have access to the 70b parameter models; however, we have not tested their performance on the LS6 dev machines. 



In [4]:
# Model names (make sure you have access on HF)
LLAMA2_7B = f"{scratch[0]}/HF/noRef/Llama7b"
LLAMA2_7B_CHAT = f"{scratch[0]}/HF/noRef/Llama7bchat"
LLAMA2_13B = "meta-llama/Llama-2-13b-hf"
LLAMA2_13B_CHAT = "meta-llama/Llama-2-13b-chat-hf"

## Select Model
For this script we will chose the Llama 2 13B parameter chat model. 

In [5]:
selected_model  =LLAMA2_7B_CHAT

## Sytem Prompt
We are going to have llama2 create triplets used in a knowledge graph based on a pdf. 

We set up the system prompt below. 


In [6]:
SYSTEM_PROMPT="""You are social scientist researcher. You have been tasked with identifying the topics of research papers. 

         
Here are some rules you always follow:
- Do not include numbers before the topics. 
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

## Load the Model
Next we'll load the model. If it can't find the model it will download it. 

In [7]:

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True, "cache_dir":f"{scratch[0]}/HF/noRef"},
)
print(selected_model)

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpwsdu6eko
Created a temporary directory at /tmp/tmpwsdu6eko
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpwsdu6eko/_remote_module_non_scriptable.py
Writing /tmp/tmpwsdu6eko/_remote_module_non_scriptable.py


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/scratch/06659/wmobley/HF/noRef/Llama7bchat


## Load the PDF documents 

In [8]:


pandas = download_loader("PandasCSVReader")

loader = pandas()


In [9]:
from llama_index import VectorStoreIndex, ServiceContext, set_global_service_context


# Better Explain Each of these steps. 
topics = {}

In [10]:
import pandas as pd
documents = pd.read_csv(f'{scratch[0]}/HF/Survey/all_public_interviews.csv')

In [11]:

for g in documents.groupby(["Interview",'Set']):
    doc_chunks= []
    for text in g[1].Text.tolist():
        doc = Document(text=text)
        doc_chunks.append(doc)
    service_context = ServiceContext.from_defaults(
        llm=llm, embed_model="local:BAAI/bge-small-en"
    )
    set_global_service_context(service_context)

    index = VectorStoreIndex.from_documents(doc_chunks)
    query_engine = index.as_query_engine()
    response = query_engine.query("Provide a list of Topics")

    response_dict = {"".join(g[0]): re.sub('\d', "" , response.response.split(":")[1].replace('\n',"")).replace('()',"").split(".")}
    topics.update(response_dict)
    print(g[0])

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/work/06659/wmobley/minconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/work/06659/wmobley/minconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


('INTERVIEW 1 ', '21-22 Combined')
('INTERVIEW 10 ', '21-22 Combined')
('INTERVIEW 11 ', '21-22 Combined')
('INTERVIEW 12 ', '21-22 Combined')
('INTERVIEW 13 ', '21-22 Combined')
('INTERVIEW 14 ', '21-22 Combined')
('INTERVIEW 15 ', '21-22 Combined')
('INTERVIEW 16 ', '21-22 Combined')
('INTERVIEW 17 ', '21-22 Combined')
('INTERVIEW 2 ', '21-22 Combined')
('INTERVIEW 3 ', '21-22 Combined')
('INTERVIEW 4 ', '21-22 Combined')
('INTERVIEW 5 ', '21-22 Combined')
('INTERVIEW 6 ', '21-22 Combined')
('INTERVIEW 7 ', '21-22 Combined')
('INTERVIEW 8 ', '21-22 Combined')
('INTERVIEW 9 ', '21-22 Combined')
('Interview 1 ', 'April 22')
('Interview 10 ', 'April 22')
('Interview 11 ', 'April 22')
('Interview 12 ', 'April 22')
('Interview 13 ', 'April 22')
('Interview 14 ', 'April 22')
('Interview 15 ', 'April 22')
('Interview 16 ', 'April 22')
('Interview 17 ', 'April 22')
('Interview 18 ', 'April 22')
('Interview 19 ', 'April 22')
('Interview 2 ', 'April 22')
('Interview 20 ', 'April 22')
('Intervi

In [12]:
print(response.response)
import json 
with open("NonCodedSurveyTopics.json", "w") as outfile: 
    json.dump(topics, outfile)

Certainly! Based on the provided context information, here are the potential topics of research papers that could be relevant to the scenario:
1. Pipeline infrastructure and management
2. Deferred revenue accounting and reporting
3. Urban planning and development strategies for water delivery systems
4. Sustainable water management practices and technologies
5. Water pipeline design and engineering
6. Water supply chain management and logistics
7. Financial modeling and forecasting for water infrastructure projects
8. Public-private partnerships in water infrastructure development
9. Water policy and regulation analysis
10. Stakeholder engagement and public participation in water infrastructure planning.


In [13]:
topics

{'INTERVIEW 1 21-22 Combined': ['',
  ' Public perceptions and attitudes towards water quality and management',
  ' Communication strategies for engaging the public in water-related decision-making processes',
  ' Stakeholder analysis and mapping for water-related projects',
  ' Public participation and involvement in water governance',
  ' Water literacy and education'],
 'INTERVIEW 10 21-22 Combined': ['',
  ' Water access and availability in tribal communities',
  ' Collaboration between tribes and health corporations in water management',
  ' Public perceptions and attitudes towards water in tribal areas',
  ' The role of traditional knowledge in water management in tribal communities',
  ' The impact of water scarcity on tribal cultures and traditions',
  ' Collaborative approaches to water management in tribal areas',
  ' The intersection of water and cultural heritage in tribal communities',
  ' The role of indigenous knowledge in water conservation and sustainability',
  ' The 